<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/PDSH-cover-small.png?raw=1">

*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

<!--NAVIGATION-->
< [Combining Datasets: Merge and Join](03.07-Merge-and-Join.ipynb) | [Contents](Index.ipynb) | [Pivot Tables](03.09-Pivot-Tables.ipynb) >

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.08-Aggregation-and-Grouping.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>


# 聚合計算與分組


本節將探討在Pandas的聚合計算，從類似於Numpy中看到的簡單運算(如``sum()``, ``mean()``, ``median()``, ``min()``和``max()``)，到以``groupby``概念為基礎的複雜運算

## 行星資料



In [ ]:
import seaborn as sns
planets = sns.load_dataset('planets') # seaborn預設的資料集
planets.shape

(1035, 6)

In [ ]:
print(planets)

               method  number  orbital_period   mass  distance  year
0     Radial Velocity       1      269.300000   7.10     77.40  2006
1     Radial Velocity       1      874.774000   2.21     56.95  2008
2     Radial Velocity       1      763.000000   2.60     19.84  2011
3     Radial Velocity       1      326.030000  19.40    110.62  2007
4     Radial Velocity       1      516.220000  10.50    119.47  2009
...               ...     ...             ...    ...       ...   ...
1030          Transit       1        3.941507    NaN    172.00  2006
1031          Transit       1        2.615864    NaN    148.00  2007
1032          Transit       1        3.191524    NaN    174.00  2007
1033          Transit       1        4.125083    NaN    293.00  2008
1034          Transit       1        4.187757    NaN    260.00  2008

[1035 rows x 6 columns]


In [ ]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


## 在Pandas中的簡單聚合運算

In [ ]:
import numpy as np
import pandas as pd
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [ ]:
ser.sum()

2.811925491708157

In [ ]:
ser.mean()

0.5623850983416314

In [ ]:
# 在Dataframe中，預設下聚合運算可以傳回每一欄的結果
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [ ]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

In [ ]:
# 以column為單位進行計算
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

In [ ]:
# 用describe輕鬆取得常見的統計值
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


Pandas內建的聚合計算摘要:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

以上是``DataFrame``和``Series``物件的所有方法

要深入了解資料，這些簡單的計算絕對不夠。

可用``groupby``的方式，有效且快速對於資料子集合進行聚合計算。

## GroupBy: Split, Apply, Combine



### Split, apply, combine



![](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/03.08-split-apply-combine.png?raw=1)
[figure source in Appendix](06.00-Figure-Code.ipynb#Split-Apply-Combine)

- *切割(split)*: 依照指定鍵的值分解和重組一個``DataFrame``
- *套用(apply)*: 在一個特定分組中計算某一個函數，通常是聚合計算、轉換或是過濾。
- *合併(combine)*: 合併之前的運算結果把它變成一個輸出的陣列。

``GroupBy``的威力在於*中間計算產生的結果不會產生出來*，使用者不需要想這些計算是怎麼形成，就直接把這些運算當作是一個完整的操作即可。


In [ ]:
# 舉例
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [ ]:
# 用groupby進行最基本的split-apply-combine運算
df.groupby('key')

傳回來的不是 ``DataFrame``的集合，而是``DataFrameGroupBy``物件。

可以把這個物件當作是``DataFrame``的特殊視角，可以讓我們深入檢視這個群組的內容，但不真正執行實際的運算，直到聚合計算被設定為止。這樣的特性稱為惰性計算(lazy evaluation)，在聚合計算很常見，而且可以很有效的處立。

In [ ]:
# 可套用聚合計算到DataFrameGroupBy物件
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


### GroupBy物件

``GroupBy``物件是一個很抽象的概念，在多數情況下，可以把它當作``DataFrame``的collection，而且所有困難的事都在看不到的地方執行。

#### 欄索引

``GroupBy``物件支援就像是在``DataFrame``中的欄索引, 而傳回值就是修改過的``GroupBy``物件。

In [ ]:
planets.groupby('method')

In [ ]:
planets.groupby('method')['orbital_period']

In [ ]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

#### 在群組中進行迭代

``GroupBy``物件支援直接在群組中進行迭代，並把每一個群組以``Series``或``DataFrame``

In [ ]:
# 寫一個迴圈把method和group.shape一口氣讀出來
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


### Aggregate, filter, transform, apply

``GroupBy``物件有``aggregate()``, ``filter()``, ``transform()``和``apply()``方法可以有效率的在合併群組資料之前實作各式各樣有用的操作。 

In [ ]:
# 先產生一個DataFrame
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


#### 聚合計算(Aggregation)

``aggregate()``方法更有彈性去做聚合計算，可以輸入字串、函式、或是一個串列，然後一次執行所有聚合計算。

In [ ]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

或是傳送一個字典，此字典可把欄名對應到該欄位要被套用的運算上

In [ ]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


#### 過濾(Filtering)

可基於資料特性丟掉一些資料

In [ ]:
def filter_func(x):
    return x['data2'].std() > 4 # 保留標準差>4的資料

display(df, df.groupby('key').std(), df.groupby('key').filter(filter_func))

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


,data1,data2
key,,
A,2.12132,1.414214
B,2.12132,4.949747
C,2.12132,4.242641


,key,data1,data2
1,B,1,0
2,C,2,3
4,B,4,7
5,C,5,9


#### 轉換(Transformation)

當聚合函式需要傳回一個資料的縮減版本，轉換回傳一些整個資料轉換過的版本送去新合併。轉換輸出和輸入的形狀會是一樣的。

In [ ]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


#### apply()

``apply()``可套用任一函式到群組結果。此函式輸入一個``DataFrame``，然後傳回一個Pandas物件(例如:``DataFrame``、``Series``)或是一個純量；合併操作會調整為適合輸出的型態。


In [ ]:
# 藉由apply函式讓第二欄的總和來正規化第一欄
def norm_by_data2(x):
    # x是群組值的DataFrame
    x['data1'] /= x['data2'].sum() # x['data1'] = x['data1']/x['data2'].sum()
    return x

display(df, df.groupby('key').apply(norm_by_data2))

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3
3,A,0.375000,3
4,B,0.571429,7
5,C,0.416667,9


#### 字典或序列對應索引到群組

另一個方法是提供一個字典對應到索引值來當作群組鍵:

In [ ]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display(df2, df2.groupby(mapping).sum())

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9


,data1,data2
key,,
consonant,12,19
vowel,3,8


#### 任何Python函式

和mapping相似，也可以傳遞任一個Python函式，它將會輸入這個索引值，然後輸出這個群組:

In [ ]:
display(df2, df2.groupby(str.upper).mean())

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9


,data1,data2
key,,
A,1.5,4.0
B,2.5,3.5
C,3.5,6.0


#### 有效鍵的list

更進一步，之前所有可以選用的鍵均可以在多索引上被合併到群組:

In [ ]:
df2.groupby([str.upper, mapping]).mean()

,,data1,data2
key,key,,
A,vowel,1.5,4.0
B,consonant,2.5,3.5
C,consonant,3.5,6.0


In [ ]:
df2.groupby([str.upper, mapping]).mean()

,,data1,data2
key,key,,
A,vowel,1.5,4.0
B,consonant,2.5,3.5
C,consonant,3.5,6.0


### 群組範例

透過幾行code，可把所有之前看到的都放在一起，以"發現的方法"和"10年為單位"計算被發現行星的數量:

In [ ]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0
